In [1]:
import sys
from sklearn import datasets
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pandas.plotting import scatter_matrix
import os
os.chdir(r"C:\Users\Mohammed Rizwan\Documents\Data Science and Machine learning journey\Jupyter notebook\Misc Projects")
%matplotlib inline

In [2]:
df_r=pd.read_csv("Annexure2_result.csv")
df_t=pd.read_csv("Annexure_1_result.csv")

In [3]:
df_r=df_r.drop(["Unnamed: 0"],axis=1)
df_t=df_t.drop(["Unnamed: 0"],axis=1)

In [4]:
df_t.head()

,Sample_number,Text_id,Sampled_date,T_site,T_plant,Sampling_point,CH4(ppm),H2(ppm),C2H2(ppm),C2H4(ppm),C2H6(ppm),CO(ppm),CO2(ppm),TDCG(ppm),Condition,Fault
0,10735333.0,OIL/NGP-DHC-SUB-55ETDC055001/011117@1159,11/28/2016 0:00,NMD,DHC_ELE_NMD,55ETDC055001_NMD,1.0,12.0,0.0,3.0,0.0,59.0,1619.0,75.0,Condition 1,No fault
1,10777576.0,OIL/NGU-IOP-ETP-SUB-80XD0011/141117@1801,11/28/2016 0:00,NMD,IWWTP_ELE_NMD,80XD0011_NMD,1.0,7.0,0.0,2.0,0.0,61.0,1634.0,71.0,Condition 1,No fault
2,10777591.0,OIL/NGU-IOP-ETP-SUB-80XD0012/141117@1801,11/28/2016 0:00,NMD,IWWTP_ELE_NMD,80XD0012_NMD,1.0,6.0,0.0,1.0,0.0,36.0,1514.0,44.0,Condition 1,No fault
3,10777595.0,OIL/NGU-IOP-ETP-SUB-80XD0013/141117@1801,11/28/2016 0:00,NMD,IWWTP_ELE_NMD,80XD0013_NMD,1.0,9.0,0.0,3.0,0.0,63.0,1172.0,76.0,Condition 1,No fault
4,10777600.0,OIL/NGU-IOP-ETP-SUB-80XD0014/141117@1801,11/28/2016 0:00,NMD,IWWTP_ELE_NMD,80XD0014_NMD,1.0,12.0,0.0,3.0,0.0,116.0,2206.0,132.0,Condition 1,No fault


In [5]:
df_r.head()

,Text_id,T_site,T_plant,Sampling_point,Rate(%)
0,['OIL/NGP-DHC-SUB-55ETDC055001/011117@1159'\r\...,['NMD'],['DHC_ELE_NMD'],55ETDC055001_NMD,0.026
1,['OIL/NGU-IOP-ETP-SUB-80XD0011/141117@1801'],['NMD'],['IWWTP_ELE_NMD'],80XD0011_NMD,0.064
2,['OIL/NGU-IOP-ETP-SUB-80XD0012/141117@1801'],['NMD'],['IWWTP_ELE_NMD'],80XD0012_NMD,0.018
3,['OIL/NGU-IOP-ETP-SUB-80XD0013/141117@1801'],['NMD'],['IWWTP_ELE_NMD'],80XD0013_NMD,0.026
4,['OIL/NGU-IOP-ETP-SUB-80XD0014/141117@1801'],['NMD'],['IWWTP_ELE_NMD'],80XD0014_NMD,0.026


In [6]:
An4_sp=df_r.Sampling_point.unique()

In [7]:
condition=""
samp_int=""
recommendation=""
def find_an3(tdcg_1,R):
    if(tdcg_1>4630):
        if(R>30|(R>10 and R<30)):
            condition='Condition 4'
            samp_int="Daily"
            recommendation="Consider Removal From Service Advice From Manufacturer."
        elif(R<10):
            condition='Condition 4'
            samp_int="Weekly"
            recommendation="Exercise extreme caution. Analyze for individual gases. Plan outrage. Advice manufacturer."
    elif(tdcg_1>1920 and tdcg_1<=4630):
        if(R>30|(R>10 and R<30)):
            condition='Condition 3'
            samp_int="Weekly"
            recommendation="Exercise extreme caution. Analyze for individual gases. Plan outrage. Advice manufacturer."
        elif(R<10):
            condition='Condition 3'
            samp_int="Monthly"
            recommendation="Exercise extreme caution. Analyze for individual gases. Plan outrage. Advice manufacturer."
    elif(tdcg_1>720 and tdcg_1<=1920):
        if(R>30|(R>10 and R<30)):
            condition='Condition 2'
            samp_int="Monthly"
            recommendation="Exercise caution Analyze for individual gases Determine Load dependence."
        elif(R<10):
            condition='Condition 2'
            samp_int="Quaterly"
            recommendation="Exercise caution Analyze for individual gases Determine Load dependence."
    elif(R<=720):
        if(R>30):
            condition='Condition 1'
            samp_int="Monthly"
            recommendation="Exercise caution Analyze for individual gases Determine Load dependence."
        elif(R>10 and R<30):
            condition='Condition 1'
            samp_int="Quaterly"
            recommendation="Exercise caution Analyze for individual gases Determine Load dependence." 
        elif(R<10):
            condition='Condition 1'
            samp_int="Annual"
            recommendation="Continue normal operation."
    return condition,samp_int,recommendation

In [42]:
an3df=pd.DataFrame({
    'Conditions':[],
    'Sampling Point':[],
    'T_site':[],
    'TDGC Levels(microlitres/litres)':[],
    'TDGC Rate(micro liter/liter/day)(%)':[],
    'Sampling interval':[],
    'Operating Procedures':[]
})

In [43]:
for samp_p in An4_sp:
    TDCG=float(df_t[df_t.Sampling_point==samp_p]['TDCG(ppm)'].iloc[0])
    R=float(df_r[df_r.Sampling_point==samp_p]['Rate(%)'])
    condition,samp_int,recommendation=find_an3(TDCG,R)
    T_site=df_t[df_t.Sampling_point==samp_p]['T_site'].iloc[0]
    an3df_test=pd.DataFrame({
    'Conditions':[condition],
    'Sampling Point':[samp_p],
    'T_site':[T_site],
    'TDGC Levels(microlitres/litres)':[TDCG],
    'TDGC Rate(micro liter/liter/day)(%)':[R],
    'Sampling interval':[samp_int],
    'Operating Procedures':[recommendation]
    })
    an3df=an3df.append(an3df_test,ignore_index=True)

In [46]:
an3df.head()

,Conditions,Sampling Point,T_site,TDGC Levels(microlitres/litres),TDGC Rate(micro liter/liter/day)(%),Sampling interval,Operating Procedures
0,Condition 1,55ETDC055001_NMD,NMD,75.0,0.026,Annual,Continue normal operation.
1,Condition 1,80XD0011_NMD,NMD,71.0,0.064,Annual,Continue normal operation.
2,Condition 1,80XD0012_NMD,NMD,44.0,0.018,Annual,Continue normal operation.
3,Condition 1,80XD0013_NMD,NMD,76.0,0.026,Annual,Continue normal operation.
4,Condition 1,80XD0014_NMD,NMD,132.0,0.026,Annual,Continue normal operation.


In [47]:
an3df.to_csv("Annexure_3_result.csv")